In [2]:

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from timm import create_model
from tqdm import tqdm

In [3]:
# Check if MPS is available, otherwise fvalback to CUDA or CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [5]:
class CampusRegionDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])  # Assuming first column is image name
        image = Image.open(img_name).convert('RGB')
        region_id = self.annotations.iloc[idx, 5] - 1  # Assuming 6th column is Region_ID (convert to 0-indexed)
        
        if self.transform:
            image = self.transform(image)
            
        return image, region_id

In [6]:
# Define data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [7]:
# Create datasets
train_dataset = CampusRegionDataset(
    csv_file="labels_train.csv",
    img_dir="images_train",
    transform=data_transforms['train']
)

val_dataset = CampusRegionDataset(
    csv_file="labels_val.csv",
    img_dir="images_val",
    transform=data_transforms['val']
)

In [8]:
# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)


In [9]:
# Get dataset sizes
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}
print(f"Train dataset size: {dataset_sizes['train']}")
print(f"Validation dataset size: {dataset_sizes['val']}")


Train dataset size: 6542
Validation dataset size: 369


In [24]:
# Load pretrained model
model = create_model('convnextv2_tiny.fcmae_ft_in1k', pretrained=True, num_classes=15)
model = model.to(device)

In [25]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [26]:
# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_acc = 0.0
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in tqdm(train_loader if phase == 'train' else val_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.float() / dataset_sizes[phase]

            # Store statistics
            if phase == 'train':
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc.cpu().numpy())
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc.cpu().numpy())
                # Update learning rate based on validation loss
                scheduler.step(epoch_loss)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save the best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'best_campus_region_model.pth')
                print(f'New best model saved with accuracy: {best_acc:.4f}')

        print()
    
    return model, history


In [61]:
# Train the model
num_epochs = 20
model, history = train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs)


Epoch 1/20
----------


100%|██████████| 205/205 [08:29<00:00,  2.49s/it]


train Loss: 0.4784 Acc: 0.8578


100%|██████████| 12/12 [00:06<00:00,  1.90it/s]


val Loss: 0.4222 Acc: 0.8808
New best model saved with accuracy: 0.8808

Epoch 2/20
----------


...

val Loss: 0.2179 Acc: 0.9593

Epoch 20/20
----------


100%|██████████| 205/205 [07:35<00:00,  2.22s/it]


train Loss: 0.0005 Acc: 1.0000


100%|██████████| 12/12 [00:05<00:00,  2.04it/s]

val Loss: 0.2192 Acc: 0.9621



In [12]:
import pandas as pd

# Load the best model weights
best_model = create_model('convnextv2_tiny.fcmae_ft_in1k', pretrained=False, num_classes=15)
best_model.load_state_dict(torch.load('final_campus_region_model.pth'))
best_model = best_model.to(device)

# Run inference on the validation set
val_preds = []
with torch.no_grad():
    for images, _ in tqdm(val_loader):
        images = images.to(device)
        outputs = best_model(images)
        preds = torch.argmax(outputs, dim=1)
        val_preds.extend(preds.cpu().numpy())

# Prepare validation part of submission (Region_ID is 1-indexed)
val_df = pd.DataFrame({
    'id': list(range(len(val_preds))),
    'Region_ID': [p + 1 for p in val_preds]
})

# Prepare dummy test part of submission (Region_ID = 1)
test_df = pd.DataFrame({
    'id': list(range(len(val_preds), 738)),
    'Region_ID': [1] * (738 - len(val_preds))
})

# Concatenate and save
final_df = pd.concat([val_df, test_df], ignore_index=True)
final_df.to_csv("2023121004_2.csv", index=False)
print("Saved submission to 2023121004_2.csv")

# Optional: Check accuracy (for your own use)
true_labels = pd.read_csv("labels_val.csv")["Region_ID"].values
correct = sum(p + 1 == t for p, t in zip(val_preds, true_labels))
accuracy = correct / len(true_labels)
print(f"Validation Accuracy: {accuracy:.4f}")


100%|██████████| 12/12 [00:07<00:00,  1.70it/s]

Saved submission to 2023121004_2.csv
Validation Accuracy: 0.9864


In [11]:
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# Load the best model weights
best_model = create_model('convnextv2_tiny.fcmae_ft_in1k', pretrained=False, num_classes=15)
best_model.load_state_dict(torch.load('final_campus_region_model.pth'))
best_model = best_model.to(device)
best_model.eval()

# ---------------------------
# Validation predictions
# ---------------------------
val_preds = []
with torch.no_grad():
    for images, _ in tqdm(val_loader, desc="Validation Inference"):
        images = images.to(device)
        outputs = best_model(images)
        preds = torch.argmax(outputs, dim=1)
        val_preds.extend(preds.cpu().numpy())

val_df = pd.DataFrame({
    'id': list(range(len(val_preds))),
    'Region_ID': [p + 1 for p in val_preds]  # Region_ID is 1-indexed
})

# ---------------------------
# Test predictions from images_test directory
# ---------------------------
test_image_dir = "images_test"
num_test_images = 369  # img_0000.jpg to img_0368.jpg

test_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

import os
import glob

# ---------------------------
# Collect all image files (any extension) and sort by number
# ---------------------------
image_files = glob.glob(os.path.join(test_image_dir, 'img_*.*'))

# Sort by numeric index extracted from filename
def extract_index(filename):
    base = os.path.basename(filename)
    num_part = os.path.splitext(base)[0].split('_')[1]
    return int(num_part)

image_files.sort(key=extract_index)

test_preds = []
with torch.no_grad():
    for idx, img_path in tqdm(enumerate(image_files), total=len(image_files), desc="Test Inference"):
        img = Image.open(img_path).convert("RGB")
        img_tensor = test_transform(img).unsqueeze(0).to(device)
        output = best_model(img_tensor)
        pred = torch.argmax(output, dim=1).item()
        test_preds.append(pred)


test_df = pd.DataFrame({
    'id': list(range(369, 369 + num_test_images)),
    'Region_ID': [p + 1 for p in test_preds]  # Region_ID is 1-indexed
})

# ---------------------------
# Concatenate and save submission
# ---------------------------
final_df = pd.concat([val_df, test_df], ignore_index=True)
final_df.to_csv("2023121004_3.csv", index=False)
print("Saved submission to 2023121004_3.csv")

# ---------------------------
# Optional: Validation accuracy
# ---------------------------
true_labels = pd.read_csv("labels_val.csv")["Region_ID"].values
correct = sum(p + 1 == t for p, t in zip(val_preds, true_labels))
accuracy = correct / len(true_labels)
print(f"Validation Accuracy: {accuracy:.4f}")


Test Inference: 100%|██████████| 369/369 [00:07<00:00, 48.62it/s]

Saved submission to 2023121004_3.csv
Validation Accuracy: 0.9864
